In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.transforms import ToTensor, Resize, Normalize
import numpy as np
import pandas as pd
from torchvision import utils
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
import time

In [2]:
def get_transforms(kind="train"):
    if kind=="train":
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(0.5),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [3]:
# prepare train/valid folders
import os
from shutil import copyfile
import random

for pth in ["../photos/train", "../photos/val"]:
    if not os.path.exists(pth):
        os.mkdir(pth)

img_src = "../photos/labelled"

train_val_split = 0.85
for name in os.listdir(img_src):
    img_path = os.path.join(img_src, name)
    imgs = os.listdir(img_path)
    for pth in [os.path.join("../photos/train", name), os.path.join("../photos/val", name)]:
        if not os.path.exists(pth):
            os.mkdir(pth)
    for i in range(len(imgs)):
#         if i <= len(imgs) * train_val_split:
        if random.random() < train_val_split:
            # place in test
            copyfile(os.path.join(img_path, imgs[i]), os.path.join("../photos/train", name, imgs[i]))
        else:
            # place in valid
            copyfile(os.path.join(img_path, imgs[i]), os.path.join("../photos/val", name, imgs[i]))
            

In [4]:
train_src = "../photos/train"
val_src = "../photos/val"

train_ds = datasets.ImageFolder(train_src, get_transforms("train"))
val_ds = datasets.ImageFolder(val_src, get_transforms("val"))

train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=4, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
from torchvision.models import resnet18
model_conv = resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
    

In [6]:
in_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Sequential(nn.Linear(in_ftrs, len(os.listdir(img_src))), nn.Sigmoid())

criterion = nn.CrossEntropyLoss()

opt = optim.Adam(model_conv.fc.parameters(), lr=1e-3)

lr_scheduler = lr_scheduler.StepLR(opt, step_size=7, gamma=0.1)

In [7]:
import copy

def train_fn(model, dataloaders, optimizer, criterion, scheduler, device, num_epochs):
    best_acc = -1
    best_model = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}", "-"*10)
        
        for phase in ["train", "val"]:
            
            if phase == 'train':
                model.train()  
            else:
                model.eval()
            
            running_loss = []
            running_acc = []
            
            for step, (imgs, lbls) in enumerate(dataloaders[phase]):
                imgs, lbls = imgs.to(device), lbls.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(imgs)
                    value, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, lbls)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                running_loss.append(loss.item() * imgs.size(0))
                running_acc.append((torch.sum(preds == lbls)/ imgs.size(0)).cpu())
                
            if phase == "train":
                scheduler.step()
                
            epoch_loss = np.mean(running_loss)
            epoch_acc = np.mean(running_acc)
            print(f"{phase}, Loss: {epoch_loss}, Acc: {epoch_acc}")
            
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model)
    return model           
            

In [8]:
dls = {"train": train_dl, "val": val_dl}
model_conv = train_fn(model_conv.to(device), dls, opt, criterion, lr_scheduler, device, 35)

Epoch 0 ----------
train, Loss: 4.154826879501343, Acc: 0.4047619104385376
val, Loss: 4.383073925971985, Acc: 0.5
Epoch 1 ----------
train, Loss: 4.002050027960823, Acc: 0.5357142686843872
val, Loss: 4.285159349441528, Acc: 0.5
Epoch 2 ----------
train, Loss: 3.8703006903330484, Acc: 0.4523809552192688
val, Loss: 3.924210548400879, Acc: 0.5
Epoch 3 ----------
train, Loss: 3.9341049307868596, Acc: 0.5357142686843872
val, Loss: 3.80667781829834, Acc: 0.5
Epoch 4 ----------
train, Loss: 3.5727114450363886, Acc: 0.6428571343421936
val, Loss: 3.722055673599243, Acc: 0.625
Epoch 5 ----------
train, Loss: 3.506202921980903, Acc: 0.738095223903656
val, Loss: 3.4492164850234985, Acc: 0.875
Epoch 6 ----------
train, Loss: 3.4525029999869212, Acc: 0.6428571343421936
val, Loss: 3.2111575603485107, Acc: 1.0
Epoch 7 ----------
train, Loss: 3.308287751106989, Acc: 0.8095238208770752
val, Loss: 3.179069757461548, Acc: 1.0
Epoch 8 ----------
train, Loss: 3.215711709998903, Acc: 0.8333333134651184
val, 

In [9]:
torch.save(model_conv.state_dict(), "../model_save/model.pth")
np.save("../model_save/class_names", train_ds.classes)